In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [3]:
toy_data = pd.read_csv('toy_dataset.csv')
toy_data.head()

,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,NaN,40367.0,No
1,2,Dallas,Male,54.0,45084.0,No
2,3,Dallas,Male,NaN,52483.0,No
3,4,Dallas,Male,40.0,40941.0,No
4,5,Dallas,Male,NaN,50289.0,No


In [4]:
toy_data.dtypes

Number       int64
City        object
Gender      object
Age        float64
Income     float64
Illness     object
dtype: object

In [5]:
# проверим есть ли пропущенные значения
toy_data.isnull().sum()

Number     0
City       0
Gender     0
Age        3
Income     0
Illness    0
dtype: int64

In [6]:
# Применение Label Encoding к колонке 'Gender'
label_encoder = LabelEncoder()
toy_data['Gender'] = label_encoder.fit_transform(toy_data['Gender'])
toy_data['City'] = label_encoder.fit_transform(toy_data['City'])
toy_data['Illness'] = label_encoder.fit_transform(toy_data['Illness'])

In [7]:
cat_temp_data = toy_data[['Age']]
cat_temp_data.head()
cat_temp_data['Age'].unique()
cat_temp_data[cat_temp_data['Age'].isnull()].shape

(3, 1)

In [10]:
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data_imp2 = imp2.fit_transform(cat_temp_data)
data_imp2
toy_data['Age'] = data_imp2

In [11]:
toy_data.head()

,Number,City,Gender,Age,Income,Illness
0,1,2,1,26.0,40367.0,0
1,2,2,1,54.0,45084.0,0
2,3,2,1,26.0,52483.0,0
3,4,2,1,40.0,40941.0,0
4,5,2,1,26.0,50289.0,0


In [12]:
# Разделение данных на признаки и целевую переменную
X = toy_data.drop(['Illness'], axis=1)  # Предполагаем, что 'Illness_No' является целевой переменной
y = toy_data['Illness']


In [13]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Создание списка моделей
models = {
    "k-NN": KNeighborsClassifier(n_neighbors=3),
    "Bagging": BaggingClassifier(n_estimators=10, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

In [15]:
# Обучение и оценка моделей
results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Вычисление метрик
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    # Для расчета R^2, MSE и MAE, будем использовать предсказанные вероятности
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = y_pred  # Если модель не поддерживает predict_proba, используем сами предсказания
    
    r2 = r2_score(y_test, y_pred_proba)
    mse = mean_squared_error(y_test, y_pred_proba)
    mae = mean_absolute_error(y_test, y_pred_proba)
    
    results[model_name] = {
        "accuracy": accuracy,
        "confusion_matrix": conf_matrix,
        "classification_report": class_report,
        "r2": r2,
        "mse": mse,
        "mae": mae
    }

c:\python\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [17]:
# Вывод результатов
for model_name, metrics in results.items():
    print(f"Модель: {model_name}")
    print(f"Точность: {metrics['accuracy']}")
    print(f"R^2: {metrics['r2']}")
    print(f"MSE: {metrics['mse']}")
    print(f"MAE: {metrics['mae']}")
    print("\n")

Модель: k-NN
Точность: 0.9045666666666666
R^2: -0.3527288227538552
MSE: 0.0986888888888889
MAE: 0.1488


Модель: Bagging
Точность: 0.9176666666666666
R^2: -0.18628164834499894
MSE: 0.08654566666666666
MAE: 0.15749


Модель: Random Forest
Точность: 0.9186333333333333
R^2: -0.09696590783737591
MSE: 0.08002959999999999
MAE: 0.15024


Модель: AdaBoost
Точность: 0.9206666666666666
R^2: -2.2879951006895056
MSE: 0.23987703795544998
MAE: 0.4897198080302325


Модель: Gradient Boosting
Точность: 0.9207666666666666
R^2: -0.0009498809123529295
MSE: 0.07302471117574497
MAE: 0.1476810437515269


